In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import nibabel as nib
import os
import time

import pandas as pd
import numpy as np

from mricode.utils import log_textfile

from mricode.utils import return_csv
#from mricode.utils import return_iter

In [3]:
path_output = './'
path_tfrecords = '/data2/res64/down/'
path_csv = '/data2/csv/'
filename_res = {'train': 'intell_residual_train.csv', 'val': 'intell_residual_valid.csv', 'test': 'intell_residual_test.csv'}
filename_final = filename_res
sample_size = 'site16_allimages'
batch_size = 8
onlyt1 = False
t1_mean = 0.35196779465675354 
t2_mean = 0.5694633522033692
t1_std = 0.8948413240464094
t2_std = 1.2991791534423829

In [4]:
train_df, val_df, test_df, norm_dict = return_csv(path_csv, filename_final, False)

In [5]:
cat_cols = {'female': 2, 'race.ethnicity': 5, 'high.educ_group': 4, 'income_group': 8, 'married': 6}
num_cols = [x for x in list(val_df.columns) if '_norm' in x]

## 01 Resolution

In [6]:
output = []
for col in num_cols:
  mean = test_df[col].mean()
  mse_norm = np.mean(np.square(test_df[col]-mean))
  output.append([col, mse_norm])

for col in list(cat_cols.keys()):
  mean = test_df[col].value_counts().idxmax()
  mse_norm = np.mean(test_df[col]==mean)
  output.append([col, mse_norm])
df = pd.DataFrame(output)
df.columns = ['colname', 'mse']
df_base = df.copy()

In [7]:
df_add = {'new_simplecnnsmall__allimages_down64': 'down64',
          'new_simplecnnsmall__allimages_cropped64': 'cropped64',
          'new_simplecnnsmall__allimages_down128': 'down128',
          'new_simplecnnsmall__allimages_cropped128': 'cropped128',
          'simplecnnsmall__allimages_down256': 'org256'
          }

In [8]:
df_add_keys = list(df_add.keys())

In [9]:
for df_name in df_add_keys:
    df = pd.read_csv('./output/' + df_name + '/df_best.csv')
    df.columns = [df_name + str(x) for x in df.columns]
    df_base = pd.merge(df_base, df, left_on='colname', right_on=df_name+'name',how='left')

In [11]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[~(aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity']))]

,colname,mse,down64,cropped64,org256,down128,cropped128
0,BMI_norm,0.930882,0.243475,0.247966,5.063574,0.223083,0.302021
1,age_norm,1.005556,1.073573,0.887765,0.867961,0.786431,0.953249
2,vol_norm,0.986314,0.073409,0.127015,0.139645,0.053141,0.128712
3,weight_norm,0.978025,0.177938,0.211520,1.097935,0.155521,0.207269
4,height_norm,1.084821,0.646673,0.657459,0.720825,0.583928,0.591562
5,nihtbx_fluidcomp_uncorrected_norm,0.992199,1.025295,1.018468,1.004899,1.008545,1.006609
6,nihtbx_cryst_uncorrected_norm,1.029535,1.045991,1.059609,1.095101,1.065390,1.025904
7,nihtbx_pattern_uncorrected_norm,0.969806,1.031810,0.993947,1.038769,0.999553,0.974088
8,nihtbx_picture_uncorrected_norm,0.916866,0.959523,0.933301,0.965138,0.952819,1.028652
9,nihtbx_list_uncorrected_norm,1.048735,1.091219,1.107307,1.136448,1.059911,1.072621


In [12]:
add_col = [x for x in df_base.columns if 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity'])]

,colname,mse,org256,cropped64,down128,down64,cropped128
15,race.ethnicity,0.531625,0.601281,0.674139,0.655725,0.690953,0.587670
16,income_group,0.290633,0.322658,0.305845,0.294636,0.300240,0.305044
17,high.educ_group,0.874299,0.874299,0.874299,0.874299,0.874299,0.874299
18,female,0.534027,0.704564,0.795036,0.867094,0.811049,0.821457
19,married,0.707766,0.708567,0.712570,0.723779,0.706966,0.726181


In [13]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[~(aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity']))].to_csv('./output_csv/01_new_resolution_num_256.csv')
add_col = [x for x in df_base.columns if 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity'])].to_csv('./output_csv/01_new_resolution_cat_256.csv')

## 02 Model Type

In [13]:
output = []
for col in num_cols:
  mean = test_df[col].mean()
  mse_norm = np.mean(np.square(test_df[col]-mean))
  output.append([col, mse_norm])

for col in list(cat_cols.keys()):
  mean = test_df[col].value_counts().idxmax()
  mse_norm = np.mean(test_df[col]==mean)
  output.append([col, mse_norm])
df = pd.DataFrame(output)
df.columns = ['colname', 'mse']
df_base = df.copy()
df_add = {'new_simplecnnsmall__allimages_down64': 'simplecnnsmall',
          'new_simplecnnnormal_allimages_down64': 'simplecnnnormal',
          'new_simplecnnnormal_autofocus_allimages_down64': 'simplecnnnormal_auto',
          'new2_densenet_allimages_down64': 'densenet',
          'new_densenet_allimages_autofocus_down64': 'densenet_auto'
          }
df_add_keys = list(df_add.keys())
for df_name in df_add_keys:
    df = pd.read_csv('./output/' + df_name + '/df_best.csv')
    df.columns = [df_name + str(x) for x in df.columns]
    df_base = pd.merge(df_base, df, left_on='colname', right_on=df_name+'name',how='left')

In [14]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[~(aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity']))].mean()

mse                     1.000309
simplecnnnormal_auto    0.793970
simplecnnnormal         0.806132
simplecnnsmall          0.840636
densenet                0.789093
densenet_auto           0.794372
dtype: float64

In [17]:
add_col = [x for x in df_base.columns if 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity'])].mean()

mse                     0.587670
simplecnnnormal_auto    0.703122
simplecnnnormal         0.697518
simplecnnsmall          0.676701
densenet                0.701201
densenet_auto           0.688711
dtype: float64

In [18]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[~(aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity']))].to_csv('./output_csv/02_new_model_type_num.csv')
add_col = [x for x in df_base.columns if 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity'])].to_csv('./output_csv/03_new_model_type_cat.csv')

## 03 MRI Types

In [11]:
output = []
for col in num_cols:
  mean = test_df[col].mean()
  mse_norm = np.mean(np.square(test_df[col]-mean))
  output.append([col, mse_norm])

for col in list(cat_cols.keys()):
  mean = test_df[col].value_counts().idxmax()
  mse_norm = np.mean(test_df[col]==mean)
  output.append([col, mse_norm])
df = pd.DataFrame(output)
df.columns = ['colname', 'mse']
df_base = df.copy()

In [12]:
df_add = {'new2_densenet_allimages_down64': 't1t2',
          'new_densenet_allimages_t1t2def_down64': 't1t2def',
          'new_densenet_allimages_t1_down64': 't1',
          'new_densenet_allimages_t2_down64': 't2',
          'new_densenet_allimages_def_down64': 'def'
          }

In [13]:
df_add_keys = list(df_add.keys())

In [14]:
for df_name in df_add_keys:
    df = pd.read_csv('./output/' + df_name + '/df_best.csv')
    df.columns = [df_name + str(x) for x in df.columns]
    df_base = pd.merge(df_base, df, left_on='colname', right_on=df_name+'name',how='left')

In [15]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[~(aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity']))].mean()

mse         1.000309
t2          0.789305
t1          0.872441
t1t2def     0.850931
t1t2        0.789093
def        13.873018
dtype: float64

In [16]:
add_col = [x for x in df_base.columns if 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity'])]

,colname,mse,t2,t1,t1t2def,t1t2,def
15,income_group,0.290633,0.314652,0.319456,0.312250,0.325861,0.286629
16,married,0.707766,0.720576,0.722178,0.714972,0.726181,0.707766
17,female,0.534027,0.805444,0.751001,0.763010,0.870296,0.534027
18,high.educ_group,0.874299,0.874299,0.874299,0.873499,0.875100,0.874299
19,race.ethnicity,0.531625,0.632506,0.698959,0.626101,0.708567,0.531625


In [17]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[~(aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity']))].to_csv('./output_csv/03_new_types_num.csv')
add_col = [x for x in df_base.columns if 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity'])].to_csv('./output_csv/03_new_types_cat.csv')

## 04 Training

In [23]:
output = []
for col in num_cols:
  mean = test_df[col].mean()
  mse_norm = np.mean(np.square(test_df[col]-mean))
  output.append([col, mse_norm])

for col in list(cat_cols.keys()):
  mean = test_df[col].value_counts().idxmax()
  mse_norm = np.mean(test_df[col]==mean)
  output.append([col, mse_norm])
df = pd.DataFrame(output)
df.columns = ['colname', 'mse']
df_base = df.copy()

In [24]:
df_add = {'new2_densenet_allimages_down64': 'desnet_t1t2',
          'new_simplecnnnormal_allimages_down64': 'simplenormal_t1t2',
          'new_cross_allimages_down64': 'cross',
          'new_hiearch_allimages_down64': 'simplenormal_hierch',
          'new_hiearch_densenet_allimages_down64': 'densenet_hierch',
          'new_densenet_cross_allimages_down64': 'cross_densenet1',
          'new2_densenet_cross_allimages_down64': 'cross_densenet2'
          }

In [25]:
df_add_keys = list(df_add.keys())

In [26]:
for df_name in df_add_keys:
    df = pd.read_csv('./output/' + df_name + '/df_best.csv')
    df.columns = [df_name + str(x) for x in df.columns]
    df_base = pd.merge(df_base, df, left_on='colname', right_on=df_name+'name',how='left')

In [27]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[~(aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity']))]

,colname,mse,cross,cross_densenet1,densenet_hierch,simplenormal_hierch,cross_densenet2,desnet_t1t2,simplenormal_t1t2
0,BMI_norm,0.930882,0.209201,0.767930,0.225635,0.371828,0.257790,0.214400,0.273162
1,age_norm,1.005556,0.872058,2.199053,0.921705,0.895678,0.974896,0.856505,0.839964
2,vol_norm,0.986314,0.050491,1.495348,0.035456,0.091716,0.267672,0.061479,0.073071
3,weight_norm,0.978025,0.180605,0.195233,0.179614,0.217069,0.189348,0.126339,0.141554
4,height_norm,1.084821,0.827486,3.201904,0.548577,0.662880,0.671672,0.523104,0.549244
5,nihtbx_fluidcomp_uncorrected_norm,0.992199,1.036300,0.996323,0.996246,0.991058,1.021275,0.995088,1.000368
6,nihtbx_cryst_uncorrected_norm,1.029535,1.053267,1.288080,1.026663,1.032484,1.053927,1.032553,1.052525
7,nihtbx_pattern_uncorrected_norm,0.969806,0.987934,1.016443,0.969088,0.989037,0.986437,0.978598,1.006918
8,nihtbx_picture_uncorrected_norm,0.916866,0.924462,0.981696,0.922632,0.916651,0.921058,0.921239,0.926470
9,nihtbx_list_uncorrected_norm,1.048735,1.088634,1.049941,1.046501,1.059709,1.086653,1.056780,1.066986


In [28]:
add_col = [x for x in df_base.columns if 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity'])]

,colname,mse,cross,cross_densenet1,densenet_hierch,simplenormal_hierch,cross_densenet2,desnet_t1t2,simplenormal_t1t2
15,married,0.707766,0.730985,0.715773,0.711769,0.726181,0.726181,0.726181,0.731785
16,income_group,0.290633,0.307446,0.305044,0.297038,0.320256,0.296237,0.325861,0.313050
17,race.ethnicity,0.531625,0.678943,0.556445,0.724580,0.660528,0.620496,0.708567,0.718175
18,female,0.534027,0.885508,0.772618,0.915933,0.778223,0.690152,0.870296,0.851081
19,high.educ_group,0.874299,0.874299,0.874299,0.874299,0.873499,0.874299,0.875100,0.873499


In [29]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[~(aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity']))].to_csv('./output_csv/04_multitask_num.csv')
add_col = [x for x in df_base.columns if 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = ['colname', 'mse'] + [df_add[x] for x in df_add_keys]
aa.loc[aa['colname'].isin(['female', 'income_group', 'high.educ_group', 'married', 'race.ethnicity'])].to_csv('./output_csv/04_multitask_cat.csv')

In [ ]:
train_df, val_df, test_df, norm_dict = return_csv(path_csv, filename_final, False)

In [ ]:
train_df['abcd_site'].value_counts()

In [ ]:
train_df = train_df[train_df['abcd_site']==16]
val_df = val_df[val_df['abcd_site']==16]
test_df = test_df[test_df['abcd_site']==16]

In [ ]:
cat_cols = {'female': 2, 'race.ethnicity': 5, 'high.educ_group': 4, 'income_group': 8, 'married': 6}
num_cols = [x for x in list(val_df.columns) if '_norm' in x]

In [ ]:
output = []
for col in num_cols:
  mean = test_df[col].mean()
  mse_norm = np.mean(np.square(test_df[col]-mean))
  output.append([col, mse_norm])

for col in list(cat_cols.keys()):
  mean = test_df[col].value_counts().idxmax()
  mse_norm = np.mean(test_df[col]==mean)
  output.append([col, mse_norm])

In [ ]:
df = pd.DataFrame(output)
df.columns = ['colname', 'mse']

In [ ]:
df_base = df.copy()

In [ ]:
df_add = ['runAllImages64_DenseNet_T1T2_site16_norm_multitask_test.csv', 
          'runAllImages64_DenseNet_T1T2_site16_onlyIntel_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_norm_20emultitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_onlyIntel_20emultitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_onlyIntel_20e_onlyIntel_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_onlyIntel_20e_SimpleCNN_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_onlyIntel_20e_onlyIntel_SimpleCNN_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_norm_20e_SimpleCNN_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_T1_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_T2_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_def_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_T1T2def_multitask_test.csv',
          ]

In [ ]:
for df_name in df_add:
    df = pd.read_csv('./output/' + df_name)
    df.columns = [df_name + str(x) for x in df.columns]
    df_base = pd.merge(df_base, df, left_on='colname', right_on=df_name+'name',how='left')

In [ ]:
df_base.head()

In [ ]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x or 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = range(0,aa.shape[1])
aa[[0,1] + [x for x in range(2,aa.shape[1],2)]].to_csv('ana1.csv')

In [ ]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x or 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = range(0,aa.shape[1])
aa[[0,1] + [x+1 for x in range(2,aa.shape[1],2)]].to_csv('ana2.csv')

In [ ]:
df_base.to_csv('output_site16.csv')

In [ ]:
test_df['female'].value_counts()

In [ ]:
66/(66+56)

In [ ]:
from tf_explain.core.smoothgrad import SmoothGrad